In [1]:
import collections
import h5py
import keras.layers
import keras.layers.advanced_activations
import keras.layers.convolutional
import keras.layers.core
import keras.layers.pooling
import keras.models
import keras.utils.np_utils
import numpy as np
import os.path

Using TensorFlow backend.


In [2]:
current_path = "/home/vl1019/tetci17/experiments/deprecated/supervised"
test_path = os.path.join(current_path, "memoized_features", "Q=08_test.mat")
hdf5_file = h5py.File(test_path)
X = hdf5_file["/"]["X_features"].value
X = np.squeeze(X)

In [29]:
y = np.concatenate([i*np.ones(10, dtype=int) for i in range(10)])
folds = np.mod(np.arange(100), 5)
n_filters = 32
kernel_size = 1
relu_alpha = 0.1


input_shape = X.shape[1:]
training_accuracies = []
test_accuracies = []

for fold_id in range(5):
    X_test = X[folds==fold_id, :, :]
    y_test = y[folds==fold_id]
    X_training = X[folds!=fold_id, :, :]
    y_training = y[folds!=fold_id]

    X_training = np.log(1e-6 + np.maximum(X_training, 0))
    X_test = np.log(1e-6 + np.maximum(X_test, 0))

    model = keras.models.Sequential()
    conv1 = keras.layers.convolutional.Conv1D(n_filters, kernel_size, input_shape=input_shape)
    model.add(conv1)
    relu1 = keras.layers.advanced_activations.LeakyReLU(alpha=relu_alpha)
    model.add(relu1)
    pool1 = keras.layers.pooling.MaxPooling1D(128 - kernel_size + 1)
    model.add(pool1)
    flatten = keras.layers.core.Flatten()
    model.add(flatten)
    dense1 = keras.layers.core.Dense(10)
    model.add(dense1)
    softmax = keras.layers.Activation("softmax")
    model.add(softmax)

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    Y_training = keras.utils.np_utils.to_categorical(y_training)
    Y_test = keras.utils.np_utils.to_categorical(y_test)
    model.fit(X_training, Y_training, epochs=25, batch_size=10, verbose=False)
    training_loss, training_accuracy = model.evaluate(X_training, Y_training, verbose=False)
    test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=False)
    training_accuracies.append(training_accuracy)
    test_accuracies.append(test_accuracy)

mean_training_accuracy = np.mean(training_accuracies)
std_training_accuracy = np.std(training_accuracies)
mean_test_accuracy = np.mean(test_accuracies)
std_test_accuracy = np.std(test_accuracies)
print("Training: " + str(100 * mean_training_accuracy) + " +/- " + str(100 * std_training_accuracy))
print("Test: " + str(100 * mean_test_accuracy) + " +/- " + str(100 * std_test_accuracy))

Training: 73.75 +/- 5.75543221661
Test: 50.0 +/- 5.47722688092


2500.0

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_67 (Conv1D)           (None, 128, 32)           43776     
_________________________________________________________________
leaky_re_lu_67 (LeakyReLU)   (None, 128, 32)           0         
_________________________________________________________________
max_pooling1d_56 (MaxPooling (None, 1, 32)             0         
_________________________________________________________________
flatten_66 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 10)                330       
_________________________________________________________________
activation_66 (Activation)   (None, 10)                0         
Total params: 44,106.0
Trainable params: 44,106.0
Non-trainable params: 0.0
_________________________________________________________________
